##Develop personalized Study Assistant
  - Reads a PDF from Google Drive and extracts the study material.
  - Generates a summary for a given topic.
  - Runs an adaptive multiple‑choice question (MCQ) assessment (10 questions total) where the question complexity starts at medium, increases when the student is correct, and decreases when the student is wrong.
  - At the end, it prints the final score.


In [ ]:
!pip install langchain==0.3.11
!pip install langchain-community==0.3.11
!pip install PyPDF2
!pip install langchain-google-genai

In [2]:
from getpass import getpass

GOOGLE_API_KEY = getpass('Please enter your Gemini Token here: ')

Please enter your Gemini Token here: ··········


In [3]:
#Environment variable
import os

os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

**PDF Parsing**

In [16]:
import PyPDF2
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

pdf_path = '/content/drive/MyDrive/Study_material.pdf'
study_material = extract_text_from_pdf(pdf_path)
display(Markdown(f"**Extracted study material length:** {len(study_material)}"))

**Extracted study material length:** 116669

# Summary Chain

In [18]:
summary_prompt = """
Act as a study assistant by providing a concise summary of the topic provided below, enclosed within triple backticks.
Utilize only the learning material provided below, enclosed within triple backticks, to generate the summary.
Ensure that your response is user-friendly and adaptable to various educational subjects.
Focus solely on the information from the learning material without introducing external content.

Format:
# {topic}
summary...

Learning Material:
```{study_material}```
Topic:
```{topic}```
"""
summary_template = ChatPromptTemplate.from_template(summary_prompt)
partial_summary_prompt = summary_template.partial(study_material=study_material)

summary_chain = (
    partial_summary_prompt
    | gemini
    | StrOutputParser()
)

def generate_summary(topic):
    result = summary_chain.invoke({"topic": topic})
    return result

# MCQ model

In [20]:
import re
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

mcq_prompt_hidden_answer = """
You are an intelligent tutor. Based solely on the study material provided, generate a {difficulty} multiple-choice question on the topic: {topic}.
Provide four answer options labeled A, B, C, and D.
**Do not** display the correct answer in the question text.
Instead, append a separate line at the end in the format:
SOLUTION: <Correct Option>

Example Format:
Q: ...
A) ...
B) ...
C) ...
D) ...
SOLUTION: B

Study Material:
```{study_material}```
"""

mcq_template_hidden_answer = ChatPromptTemplate.from_template(mcq_prompt_hidden_answer)
partial_mcq_prompt_hidden = mcq_template_hidden_answer.partial(study_material=study_material)

mcq_chain_hidden = (
    partial_mcq_prompt_hidden
    | gemini
    | StrOutputParser()
)

# -------------------------------------------------------------------
# 2. Generate and Parse the MCQ Without Displaying the Correct Answer
# -------------------------------------------------------------------
def generate_mcq_hidden(topic, difficulty):
    """Generates an MCQ with the correct answer hidden in a separate line."""
    result = mcq_chain_hidden.invoke({"topic": topic, "difficulty": difficulty})
    return result

def parse_mcq_hidden(mcq_text):
    """
    We look for the 'SOLUTION: X' line to extract the correct answer
    but do not display it to the user.
    """

    # Split on 'SOLUTION:'
    parts = mcq_text.split("SOLUTION:")
    if len(parts) < 2:
       return mcq_text, None

    question_text = parts[0].strip()
    correct_line = parts[1].strip()

    match = re.search(r"([A-D])", correct_line, re.IGNORECASE)
    correct_answer = match.group(1).upper() if match else None

    return question_text, correct_answer

# -------------------------------------------------------------------
# 3. Ask MCQ Function
# -------------------------------------------------------------------
def ask_mcq_hidden(topic):
    difficulty_levels = ['easy', 'medium', 'hard']
    current_difficulty = 1  # start at medium
    score = 0

    mcq_results = []

    for i in range(10):
        question_info = f"\n**Question {i+1} (Difficulty: {difficulty_levels[current_difficulty]})**"

        mcq_text = generate_mcq_hidden(topic, difficulty_levels[current_difficulty])
        question_text, correct_answer = parse_mcq_hidden(mcq_text)

        # store the question + answer in Markdown format
        question_block = f"{question_info}\n\n{question_text}"

        if not correct_answer:
            # If the correct answer can't be parsed, we skip
            error_msg = f"{question_block}\n\n**Error:** Could not parse the correct answer. Skipping this question."
            mcq_results.append(error_msg)
            continue

        # Print question to console (for the interactive input)
        print(question_block)
        student_answer = input("Your answer (A/B/C/D): ").strip().upper()

        if student_answer == correct_answer:
            feedback = f"**Correct!**"
            score += 1
            if current_difficulty < 2:
                current_difficulty += 1
        else:
            feedback = f"**Incorrect!** The correct answer was **{correct_answer}**."
            if current_difficulty > 0:
                current_difficulty -= 1

        # Combine question and feedback in one Markdown message
        mcq_results.append(f"{question_block}\n\n{feedback}")

    # Return both the collected messages and the final score
    return mcq_results, score

# -------------------------------------------------------------------
# 4. Integrate into Existing Pipeline
# -------------------------------------------------------------------

from IPython.display import Markdown, display
def study_assessment_pipeline(pdf_path, topic):
    display(Markdown("## 📚 Hello there! Am your Study Assistant!"))

    # display(Markdown("### 🔍 Generating Summary on (topic)"))
    summary = generate_summary(topic)
    display(Markdown(f"### 📝 **Summary on {topic}**\n{summary}"))

    display(Markdown("### 🎯 Starting MCQ Assessment..."))
    mcq_results, final_score = ask_mcq_hidden(topic)

    display(Markdown(f"### 🏆 **Final Score:** {final_score}/10"))

# Now run your pipeline as usual:
study_assessment_pipeline(pdf_path, "Question Answering")


## 📚 Hello there! Am your Study Assistant!

### 📝 **Summary on Question Answering**
# Question Answering

Natural Language Processing (NLP) in education uses techniques like question answering (QA) to help students and teachers.  Two main subcategories are Textbook Question Answering (TQA) and Math Word Problem (MWP) solving.  TQA involves understanding multimodal information (text, images, diagrams) from textbooks to answer questions, while MWP solving translates narrative descriptions into mathematical expressions.  Challenges include comprehending complex questions and contexts, and bridging the gap between natural language and abstract representations.  Solutions involve deep learning methods (RNNs, CNNs), attention mechanisms, and pre-training with domain-specific knowledge.  Large Language Models (LLMs) are increasingly used, often with strategies like Chain-of-Thought prompting to improve reasoning and zero/few-shot capabilities.  Datasets like SCIENCE QA and GSM8K are used for evaluation.

### 🎯 Starting MCQ Assessment...


**Question 1 (Difficulty: medium)**

Q:  Based on the provided study material, which of the following is NOT identified as a primary NLP application within the education domain?

A) Question Construction
B) Automated Assessment
C) Sentiment Analysis
D) Error Correction
Your answer (A/B/C/D): a

**Question 2 (Difficulty: easy)**

Q: Which of the following is NOT a sub-category of Question Answering in the education domain, as defined in the provided text?

A) Textbook Question Answering
B) Math Word Problem Solving
C) Question Generation
D) Medical Question Answering
Your answer (A/B/C/D): c

**Question 3 (Difficulty: medium)**

Q:  Based on the provided study material, which of the following is NOT identified as a primary NLP application within the education domain?

A) Question Construction
B) Automated Assessment
C) Sentiment Analysis
D) Error Correction
Your answer (A/B/C/D): d

**Question 4 (Difficulty: easy)**

Q: Which of the following is NOT a sub-category of Question Answering

### 🏆 **Final Score:** 3/10